In [1]:
import torch
from torch import nn as nn
import torch.nn.functional as F
import numpy as np
import csv
import pandas as pd
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from matplotlib import cm

In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
num_of_devices = torch.cuda.device_count()
print(f' num of devices is {num_of_devices}')
if torch.cuda.is_available():
  print(f' devive name is {torch.cuda.get_device_name()}')

 num of devices is 0


In [ ]:
### generacja danych
fun = lambda x, y: 2*np.sin( 2.8*x *y) + 3 * np.sin(2.6*x) + 2 * np.cos(4.1*y)

lb = 0
ub = 1
xx = np.arange(lb, ub, 1e-1)
xx, yy = np.meshgrid(xx, xx)

# tworzy 100 wartosci funkcji
ff = fun(xx, yy)

# prosty wykres z wykorzystaniem pyplot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xx, yy, ff, cmap=cm.coolwarm)
ax.scatter(xx, yy, ff + 0.5 * np.random.randn(*ff.shape))

In [ ]:
def generate_data(fun, noise_amp, num_of_data):
  
  mat = np.zeros((num_of_data, 3))
  for i in range(num_of_data):
    x, y = np.random.randn(2)
    # dodaje szum do danych
    f = fun(x, y) + noise_amp * np.random.randn()
    mat[i, 0] = x
    mat[i, 1] = y
    mat[i, 2] = f
  
  return mat

In [ ]:
# generate data
data_array = generate_data(fun, 0.5, 10000)

# export to csv with pandas
headers = ['x_value', 'y_value', 'fun_value']

with open('/data.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(headers)
  for i in range(data_array.shape[0]):
    writer.writerow(data_array[i, :])

In [ ]:
# load data from csv with Pandas
df = pd.read_csv('/data.csv')
print(type(df))

# show header of dataframe
df.head()

In [ ]:
# get one row
data_row = df.iloc(1)
print(df.iloc[1])

In [ ]:
# normalizacja 
arr = np.array(df)
print(arr.shape)
data_mu = np.mean(arr, axis=0)
data_std = np.std(arr, axis=0)
print(f'dama mu is : {data_mu} data std is {data_std}')

In [ ]:
### normalizing
arr = arr - data_mu
print(np.mean(arr, axis=0))
print(f'shape of data is  : {arr.shape}')
data_ar = (arr - data_mu) / (data_std**2)
print(data_ar is None)

In [ ]:
# create custom dataloader
from torch.utils.data import Dataset

class MyDataloader(Dataset):
  def __init__(self, csv_file, transform=True):
    super().__init__()

    self.data_df = pd.read_csv(csv_file)
    self.data_array = np.array(self.data_df)
    self.transform = transform
    self.data_std = np.mean(self.data_array, axis=0)
    self.data_mu = np.mean(self.data_array, axis=0)



  def __len__(self):
    return len(self.data_df)

  def __getitem__(self, index):
    
    # normalize function value to [0, 1] intervale
    if self.transform:
      self.data_array[:, -1] = (self.data_array[:, -1] - np.min(self.data_array[:, -1])) \
       / (np.max(self.data_array[:, -1]) - np.min(self.data_array[:, -1]))

    return (torch.tensor(self.data_array[index, 0:-1], dtype=torch.float32).to(device),
            torch.tensor(self.data_array[index, -1], dtype=torch.float32).to(device))

In [ ]:
# create DL
dataset = MyDataloader('/data.csv', )

In [ ]:
print(next(iter(dataset)))
print(len(dataset))

In [ ]:
train_set, test_set = torch.utils.data.random_split(dataset,
                                                    [int(0.7 * len(dataset)), int(0.3 * len(dataset))])

In [ ]:
print(f'lenght of tarinset is {len(train_set)}')
print(next(iter(train_set)))

In [ ]:
train_loader = DataLoader(dataset=train_set, batch_size = 10, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size = 10, shuffle=True)

In [ ]:
loader_output = next(iter(train_loader))
print(f'outpot of data loader {loader_output}')
print(f'shape of output of data loader {loader_output[0].shape}')
print(len(loader_output))

In [ ]:
class Regresor(nn.Module):
  def __init__(self, input_features):
    super().__init__()

    self.first_layer = nn.Linear(in_features=input_features, out_features=128)
    self.second_layer = nn.Linear(in_features=128, out_features=128)
    
    self.mid1 =  nn.Linear(in_features=128, out_features=128)
    self.mid2 = nn.Linear(in_features=128, out_features=128)

    self.last_layer = nn.Linear(in_features=128, out_features=1)
    # self.regularization_layer = nn.Dropout(p=0.5)
    self.activate = F.relu
    self.loss = F.mse_loss

    # regularization layer
    self.reg_drop = nn.Dropout(p=0.5)

  def forward(self, x):
    x = self.first_layer(x)
    x = self.activate(x)

    # x = self.regularization_layer(x)
    x = self.second_layer(x)
    x = self.activate(x)
    # regularizer
    x = self.reg_drop(x)
    x = self.mid1(x)
    x = self.activate(x)
    # regularizer
    x = self.reg_drop(x)
    x = self.mid2(x)
    x = self.activate(x)
    
    return self.last_layer(x)

In [ ]:
# tworze model
model = Regresor(input_features=2)
# device = 'cuda:0'
# sent model to cuda
if torch.cuda.is_available():
  torch.cuda.set_device(device)
  model.cuda()


# test of forward pass
data_batch, targets_batch = next(iter(train_loader))
data_batch = data_batch.to(device)
targets_batch = targets_batch.to(device)
print(data_batch.device)
print(targets_batch.device)

print(f' shape of input {data_batch.shape}')
y = model(data_batch)
print(f'data_batch is {data_batch}')
print(f'shape of output {y.shape} the type is {type(y)} and values are: {y}')

In [ ]:
# check on test set
def test_model(trained_model, is_regularize, test_loader):
  trained_model.eval()
  test_related_error = []
  test_losses = []
  
  with torch.no_grad():
    for batch_ind, (data, targets) in enumerate(test_loader):

      if torch.cuda.is_available():
        data = data.to(device)
        targets = targets.to(device)
      
      y = torch.squeeze(trained_model(data))
      
      if is_regularize:
        l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        loss = model.loss(predictions, targets) + l2_lambda * l2_norm
      
      else:
        loss = trained_model.loss(y, targets)

      ls = loss.item()
      test_losses.append(ls)

      test_related_error.append(torch.mean((targets - y) / targets).cpu())
    
    tre = torch.tensor(np.array(test_related_error))
    test_loss = torch.mean(torch.tensor(np.array(test_losses)))
    
    return test_loss, torch.mean(tre)

In [ ]:
# training loop
from tqdm import tqdm
num_of_epochs = 30
reg = False
l2_lambda = 0.001
lr = 0.00005

model = Regresor(input_features=2)

# sent model to device
if torch.cuda.is_available():
  model.cuda()

optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

test_relative_error = []
train_losses = []
test_losses = []


for epoch in tqdm(range(num_of_epochs), desc='training progress'):

  # set train mode
  model.train()

  epoch_losses = []
  for batch_index, (data, targets) in enumerate(train_loader):
    # if torch.cuda.is_available():
    data = data.to(device)
    targets = targets.to(device)

    # forward
    ### tzreba squeeze by miec te same wymiary 'target' oraz 'predict' inaczej 'loss' liczy sie zle!!
    predictions = torch.squeeze(model(data))
    if reg:
      l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
      loss = model.loss(predictions, targets) + l2_lambda * l2_norm
    
    else:
      loss = model.loss(predictions, targets)


    epoch_losses.append(loss.item()) 
    # losses.append()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  #### and of epoch ###

  # perform test after training epoch
  test_loss, test_rel_error = test_model(model,
                                             is_regularize=reg,
                                             test_loader=test_loader
                                             )

  ## save training loss
  averaged_loss = np.mean(epoch_losses)
  train_losses.append(averaged_loss)
  test_losses.append(test_loss)
  test_relative_error.append(test_rel_error)


  # compute average loss
  print(f'train loss : {averaged_loss}; test loss : {test_loss}')

In [ ]:
# print training loss

x = np.arange(0, len(train_losses))
plt.plot(x, train_losses, label='train loss')
plt.plot(x, test_losses, label='test loss')
plt.legend()
plt.grid()
plt.title(f'MSE-loss - L2 -regularisation is {reg}')